# Overview
This notebook implements multiple signal processing based data transformations to prepare bird song audio data as input to a convolutional neural network classifier. The three methods are the baseline, local energy masking, and global energy masking.

# Import

In [1]:
import librosa
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import tensorflow as tf
import h5py
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers, models
import math
from scipy.io.wavfile import write
from scipy.signal import iirfilter, sosfilt

2024-12-06 01:06:36.678268: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-06 01:06:36.712208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733447196.730241    5941 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733447196.734981    5941 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 01:06:36.780045: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Make sure GPU is available
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Settings

In [3]:
extract = False

# Extract
In this step we will extract and resample a portion of each observation, saving the resulting signals to a common HDF5 file which we can access later.

- Choose a cutoff minimum duration $s$. This will be the common duration for all the samples 
- Extract the signal and the sample rate for each sample
- Average the channels if there are multiple
- Truncate to the first $s$ seconds for each sample
- Resample to a common rate
- Save the signals and the labels to a dataset

In [4]:
# Extract metadata
metadata = pd.read_csv('/data/recordings.csv')
metadata['length'] = metadata['length'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
metadata.head()

,id,gen,sp,ssp,group,en,rec,cnt,loc,lat,...,bird-seen,animal-seen,playback-used,temp,regnr,auto,dvc,mic,smp,label
0,936105,Branta,bernicla,NaN,birds,Brant Goose,Arjun Dutta,United Kingdom,"Great Britain (near Wallington), Greater Lond...",51.3532,...,no,no,no,NaN,NaN,no,Tascam DR-05x,NaN,44100,goose
1,934302,Branta,bernicla,NaN,birds,Brant Goose,Mats Olsson,Sweden,"Dalgången 23, Karlshamn, Blekinge län",NaN,...,no,no,no,NaN,NaN,no,Wildlife Acoustics,NaN,24000,goose
2,906056,Branta,bernicla,NaN,birds,Brant Goose,Juha Saari,Finland,"Pihlajamäki, Helsinki, Uusimaa",60.2357,...,no,no,no,NaN,NaN,yes,NaN,Telinga PRO-X,44100,goose
3,898133,Branta,bernicla,NaN,birds,Brant Goose,Paul Kelly,Ireland,"Tacumshin Lake (East End), County Wexford",52.1963,...,no,no,no,NaN,NaN,yes,SM4,SM4,44100,goose
4,882013,Branta,bernicla,NaN,birds,Brant Goose,Jean COURTIN,France,"Arrondissement de Vannes (near Sérent), Morbi...",47.7981,...,yes,yes,no,NaN,NaN,yes,NaN,NaN,44100,goose


In [5]:
def extract_middle(signal, fs, duration_seconds=10):
    duration_samples = duration_seconds * fs
    signal_length = len(signal)
    start = (signal_length - duration_samples) // 2
    end = start + duration_samples
    return signal[start:end]

def extract_beginning(signal, fs, duration_seconds=10):
    duration_samples = duration_seconds * fs
    return signal[0:duration_samples]

def make_label_dict(metadata):
    return {label: i for i, label in enumerate(metadata['label'].unique())}

def load_batch(batch, label_dict, duration_seconds=10, target_fs=16000, extract_fn=extract_middle):
    signals = []
    labels = []
    ids = []
    for i, file in enumerate(batch):
        # Check that file exists
        if not os.path.exists(f'/data/recordings/{file}'):
            print(f'Skipping {file} due to missing file')
            continue
        # Load
        signal, fs = librosa.load(f'/data/recordings/{file}', sr=None, mono=False)

        # Check for multiple channels
        if signal.ndim > 1:
            signal = np.mean(signal, axis=0)

        # Extract signal of common duration
        signal = extract_fn(signal, fs, duration_seconds)
        if len(signal) != duration_seconds * fs:
            print(f'Skipping {file} due to incorrect duration')
            continue

        # Resample
        signal = librosa.resample(signal, orig_sr=fs, target_sr=target_fs)

        # Add to batch
        signals.append(signal)

        # Extract label
        file_name = file.split('_')
        labels.append(label_dict[file_name[0]])
        ids.append(int(file_name[1].split('.')[0]))

    return np.array(signals), np.array(ids), np.array(labels)

def save_batch(batch, save_path, dataset_name='signals'):
    with h5py.File(save_path, 'a') as f:
        if dataset_name in f:
            dataset = f[dataset_name]
            dataset.resize(dataset.shape[0] + batch.shape[0], axis=0)
            dataset[-batch.shape[0]:] = batch
        else:
            maxshape = (None,) + batch.shape[1:]
            f.create_dataset(dataset_name, data=batch, maxshape=maxshape, chunks=True)

def get_batches(file_list, batch_size):
    for i in range(0, len(file_list), batch_size):
        yield file_list[i:i + batch_size]

def extract_signals(file_list, save_path, label_dict, duration_seconds=10, batch_size=32, target_fs=16000, extract_fn=extract_middle):
    for i, batch_files in enumerate(get_batches(file_list, batch_size)):
        signals, ids, labels = load_batch(batch_files, label_dict, duration_seconds, target_fs, extract_fn)
        save_batch(signals, save_path, f'signals_{duration_seconds}s_{target_fs}hz')
        save_batch(labels, save_path, 'labels')
        save_batch(ids, save_path, 'ids')
        print(f'Saved {len(batch_files)} signals for batch {i}')

In [6]:
label_dict = make_label_dict(metadata)
files = os.listdir('/data/recordings')
output_path = '/data/signals.h5'

if extract:
    extract_signals(files, output_path, label_dict, duration_seconds=10, batch_size=64, target_fs=16000, extract_fn=extract_beginning)

# Transform
- Framing
- Windowing (using a Hamming function)
- Low-pass filter
- FFT (real-valued) to produce the spectrogram
- Take the log magnitude of the spectrum
- Normalize

## Baseline

In [7]:
def low_pass_filter(signal, cutoff=6250, order=8, fs=16000):
    # Design
    sos = iirfilter(
        N=order,
        Wn=cutoff / (fs / 2),
        btype='low', 
        ftype='butter',
        output='sos'
    )
    
    # Apply
    filtered_signal = sosfilt(sos, signal)
    
    return tf.convert_to_tensor(filtered_signal, dtype=tf.float32)

def tf_low_pass_filter(signal, fs=16000, filter_order=8, filter_cutoff=6250):
    result = tf.numpy_function(
        func= lambda signals: low_pass_filter(signals, cutoff=filter_cutoff, order=filter_order, fs=fs), 
        inp=[signal], 
        Tout=tf.float32
    )
    result.set_shape(signal.shape)
    return result

In [8]:
def frame_window_fft_log_dataset(dataset, frame_size = 1024, step_size = 512, fs=16000, filter_order=8, filter_cutoff=6250):
    window = tf.signal.hamming_window(frame_size)
    
    # Framing
    dataset = dataset.map(
        lambda signals, labels: (tf.signal.frame(signals, frame_size, step_size), labels),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    # Windowing
    dataset = dataset.map(
        lambda signals, labels: (signals * window, labels),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    # Apply filter
    dataset = dataset.map(
        lambda signals, labels: (tf_low_pass_filter(signals, fs, filter_order, filter_cutoff), labels),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    # Real-valued Fourier Transform
    dataset = dataset.map(
        lambda signals, labels: (tf.abs(tf.signal.rfft(signals)), labels),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    # Log-magnitude
    dataset = dataset.map(
        lambda signals, labels: (tf.math.log(signals + 1e-6), labels),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    return dataset

## Local Energy Masking

In [9]:
def mask_signals_local(ft, threshold=0.2):
    # Create mask
    row_means, row_vars = tf.nn.moments(ft, axes=[2])
    mask = tf.add(row_means, threshold * row_vars)
    mask_expanded = tf.expand_dims(mask, axis=-1)  # Shape (n, N, 1)

    # Broadcast the mask to the same shape as the spectrograms
    mask_expanded = tf.broadcast_to(mask_expanded, tf.shape(ft))  # Shape (n, N, K)

    # Apply mask
    ft_masked = tf.where(ft < mask_expanded, tf.zeros_like(ft), ft)
    return ft_masked

In [10]:
def apply_local_masking_dataset(dataset, threshold=0.2):
    # Apply mask
    dataset = dataset.map(
        lambda signals, labels: (mask_signals_local(signals, threshold), labels),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    return dataset

## Global Energy Masking

In [11]:
def mask_signals_global(ft, threshold = 0.2, cell_height=32, cell_width=32):
        # We need to reshape each sample into smaller cells
    # The size may not be consistent so we need padding
    N, K = tf.shape(ft).numpy()[1:]
    pad_N = (cell_height - (N % cell_height)) % cell_height
    pad_K = (cell_width - (K % cell_width)) % cell_width
    padding = [[0, 0], [0, pad_N], [0, pad_K]]
    ft_padded = tf.pad(ft, padding)

    # New dimensions after padding
    new_N = N + pad_N
    new_K = K + pad_K

    # Reshape to create cells
    n_cells = new_N // cell_height
    k_cells = new_K // cell_width

    # Reshape tensor into cells of size (n, n_cells, k_cells, cell_height, cell_width)
    cells = tf.reshape(ft_padded, (num_samples, n_cells, cell_height, k_cells, cell_width))

    # Compute global mean and variance
    global_mean, global_variance = tf.nn.moments(ft, axes=[1, 2])
    global_mask = global_mean + threshold * tf.sqrt(global_variance)

    # Compute means of each cell
    cell_means = tf.reduce_mean(cells, axis=[2, 4])

    # Expand the global mask to the shape of cell means
    global_mask_expanded = tf.reshape(global_mask, (num_samples, 1, 1))
    global_mask_expanded = tf.broadcast_to(global_mask_expanded, tf.shape(cell_means))

    # # Filter cells based on the global mask
    filtered_cell_means = tf.where(cell_means < global_mask_expanded, tf.zeros_like(cell_means), cell_means)

    # Expand filtered cell means back to the shape of padded tensor
    filtered_cells_expanded = tf.reshape(filtered_cell_means, (num_samples, n_cells, 1, k_cells, 1))
    filtered_cells_expanded = tf.broadcast_to(filtered_cells_expanded, tf.shape(cells))

    # # Apply the filtered cell means to the original cells
    filtered_cells = tf.where(filtered_cells_expanded == 0, tf.zeros_like(cells), cells)

    # # Reshape back to the original padded shape (n, new_N, new_K)
    padded_result = tf.reshape(filtered_cells, (num_samples, new_N, new_K))

    # # Optionally: Remove the padding to restore the original dimensions
    final_result = padded_result[:, :N, :K]
    return final_result

In [12]:
def apply_global_masking_dataset(dataset, threshold=0.2, cell_height=32, cell_width=32):
    # Apply mask
    dataset = dataset.map(
        lambda signals, labels: (mask_signals_global(signals, threshold, cell_height, cell_width), labels),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    return dataset

## Normalization

In [13]:
def normalize_dataset(dataset, num_labels=3):
    # Normalization
    dataset = dataset.map(
        lambda signals, labels: (tf.math.divide(signals, tf.reduce_max(signals)), labels),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    # OHE
    dataset = dataset.map(
        lambda signals, labels: (signals, tf.one_hot(labels, depth=num_labels)),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    dataset = dataset.map(
        lambda signals, labels: (tf.expand_dims(signals, axis=-1), labels),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    return dataset

# Create Datasets

In [14]:
# Load the signals
with h5py.File('/data/signals.h5', 'r') as f:
    signals = f['signals_10s_16000hz'][:]
    labels = f['labels'][:]

In [15]:
# Save as wav files
# if extract:
# for i, (signal, label) in enumerate(zip(signals, labels)):
#     write(f'/data/short-recordings/{i}_{label}.wav', 16000, signal)

In [16]:
def create_dataset(signals, labels):
    signal_dataset = tf.data.Dataset.from_tensor_slices(signals)
    label_dataset = tf.data.Dataset.from_tensor_slices(labels)
    dataset = tf.data.Dataset.zip((signal_dataset, label_dataset))
    dataset = dataset.batch(batch_size).prefetch(buffer_size=32)
    return dataset

def create_baseline_dataset(dataset, frame_size = 1024, step_size = 512, fs=16000, filter_order=8, filter_cutoff=6250, batch_size=64, num_labels=3):
    dataset = frame_window_fft_log_dataset(dataset, frame_size, step_size, fs, filter_order, filter_cutoff)
    dataset = normalize_dataset(dataset, num_labels=num_labels)
    return dataset

def create_local_masking_dataset(dataset, frame_size = 1024, step_size = 512, fs=16000, filter_order=8, filter_cutoff=6250, threshold=0.2, batch_size=64, num_labels=3):
    dataset = frame_window_fft_log_dataset(dataset, frame_size, step_size, fs, filter_order, filter_cutoff)
    dataset = apply_local_masking_dataset(dataset, threshold)
    dataset = normalize_dataset(dataset, num_labels=num_labels)
    return dataset

def create_global_masking_dataset(dataset, frame_size = 1024, step_size = 512, fs=16000, filter_order=8, filter_cutoff=6250, threshold=0.2, cell_height=32, cell_width=32):
    dataset = frame_window_fft_log_dataset(dataset, frame_size, step_size, fs, filter_order, filter_cutoff)
    dataset = apply_global_masking_dataset(dataset, threshold)
    dataset = normalize_dataset(dataset, num_labels=num_labels)
    return dataset

## Train/Test/Validation Split

In [17]:
train_frac = 0.7
val_frac = 0.1
test_frac = 0.2

In [18]:
sample_rate = 16000
frame_size = int(0.05 * sample_rate)
step_size = frame_size // 4
filter_order = 8
filter_cutoff = 6250
threshold = 0.2
cell_height = 32
cell_width = 32
batch_size = 8

In [19]:
def train_test_val_split(signals, labels, train_frac=0.7, val_frac=0.1, test_frac=0.2):
    # Shuffle
    indices = np.arange(len(signals))
    np.random.shuffle(indices)
    signals = signals[indices]
    labels = labels[indices]

    # Split
    train_idx = int(train_frac * len(signals))
    val_idx = int((train_frac + val_frac) * len(signals))

    train_signals = signals[:train_idx]
    val_signals = signals[train_idx:val_idx]
    test_signals = signals[val_idx:]

    train_labels = labels[:train_idx]
    val_labels = labels[train_idx:val_idx]
    test_labels = labels[val_idx:]

    return train_signals, val_signals, test_signals, train_labels, val_labels, test_labels

In [20]:
train_signals, val_signals, test_signals, train_labels, val_labels, test_labels = train_test_val_split(signals, labels, train_frac, val_frac, test_frac)

In [33]:
train_dataset = create_dataset(train_signals, train_labels)
val_dataset = create_dataset(val_signals, val_labels)
# test_dataset = create_dataset(test_signals, test_labels)

I0000 00:00:1733447296.420195    5941 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5520 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [22]:
# baseline_train_dataset = create_baseline_dataset(train_dataset, frame_size, step_size, sample_rate, filter_order, filter_cutoff, batch_size, 3)
# baseline_val_dataset = create_baseline_dataset(val_dataset, frame_size, step_size, sample_rate, filter_order, filter_cutoff, batch_size, 3)
# baseline_test_dataset = create_baseline_dataset(test_dataset, frame_size, step_size, sample_rate, filter_order, filter_cutoff, batch_size, 3)

In [23]:
# local_masking_dataset = create_local_masking_dataset(dataset, frame_size, step_size, sample_rate, filter_order, filter_cutoff, threshold)

In [24]:
# global_masking_dataset = create_global_masking_dataset(dataset, frame_size, step_size, sample_rate, filter_order, filter_cutoff, threshold, cell_height, cell_width)

# Visualize

In [25]:
def plot_spectrograms(spectrograms, labels, title, save=False):
    fig, axs = plt.subplots(8, 4, figsize=(24, 16))
    fig.suptitle(title, fontsize=20)
    for i, ax in enumerate(axs.flat):
        ax.imshow(tf.transpose(spectrograms[i, :, :]))
        ax.set_title(f'Signal {i} - {labels[i]}')
        
        # Turn of axis labels
        ax.axis('off')
    if save:
        plt.savefig(f'/data/spectrograms/{title}.png')
    else:
        plt.show()

def plot_spectrogram(spectrogram, title, save=False):
    plt.figure(figsize=(10, 4))
    plt.imshow(np.flip(tf.transpose(spectrogram), axis=0))
    plt.title(title)
    plt.xlabel('Sample')
    plt.ylabel('Frequency')
    plt.tight_layout()
    if save:
        plt.savefig(f'/data/spectrograms/{title}.png')
    else:
        plt.show()

In [26]:
# signal_batch, label_batch = next(iter(baseline_train_dataset))
# signal_batch.shape, label_batch.shape

In [27]:
# plot_spectrograms(signal_batch, label_batch, 'Baseline Spectrograms', save=False)

# Model Definition

In [28]:
def build_model(input_shape, num_classes):
    model = models.Sequential([
        # layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

    return model

In [29]:
def compute_input_size(sample_rate, duration_seconds, frame_size, step_size):
    signal_length = sample_rate * duration_seconds
    
    # Compute the number of frames
    num_frames = math.floor((signal_length - frame_size) / step_size) + 1
    
    # Compute the number of frequency bins
    frequency_bins = frame_size // 2 + 1
    
    return num_frames, frequency_bins

# Training

In [30]:
num_frames, freq_bins = compute_input_size(sample_rate, 10, frame_size, step_size)
num_frames, freq_bins

(797, 401)

In [31]:
# model = build_model((num_frames, freq_bins, 1), 3)

In [32]:
# model.fit(baseline_train_dataset, batch_size=batch_size, epochs=10)